In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("..")
sys.path.append("../..")

import json
from pathlib import Path
import numpy as np
import xarray as xr
import pandas as pd

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd

from ast import literal_eval
from typing import List
from functools import reduce

from src.paper_analysis_hydropower import NationalAnalysisHydropower, plot_pre_post_bias_correction_validation, blue

In [ ]:
cm = 1 / 2.54

In [ ]:
paths = json.load(open("../paths.json"))

path_data = Path(paths["path_data"])

path_data_hydro = path_data / "hydropower"
path_data_polygons = path_data / "polygons"
path_swiss_maps = path_data / "maps" / "swissboundaries3d_2023-01_2056_5728/"

path_figs = Path(paths["path_figs"])

In [ ]:
gdf_switzerland = gpd.read_file(path_swiss_maps / "swissBOUNDARIES3D_1_4_TLM_LANDESGEBIET.shp").to_crs("EPSG:2056")

## Load BAFU polygons
gdf_polygons = gpd.read_file(path_data_polygons / "EZG_Gewaesser.gpkg")

## Load collocated hydropower locations and BAFU polygons
df_hydropower_polygons = pd.read_json(path_data_hydro / "hydropower_polygons" / "df_hydropower_polygons.json", orient="records")
gdf_hydropower_polygons = gpd.GeoDataFrame(df_hydropower_polygons, geometry=gpd.points_from_xy(df_hydropower_polygons["_x"], df_hydropower_polygons["_y"]), crs="EPSG:2056")

## Load hydropower characteristics
df_wasta = pd.read_excel(path_data_hydro / "stats_hydropower_ch" / "wasta_2023_updated.xlsx")

## Load hydropower production (with simplified efficiency and with/without beta coefficient)
ds_hydropower_generation = xr.open_dataset(path_data_hydro / "hydropower_generation" / "ds_prevah_500_hydropower_production_ror_simplified_efficiency.nc")
ds_hydropower_generation_beta = xr.open_dataset(path_data_hydro / "hydropower_generation" / "ds_prevah_500_hydropower_production_ror_simplified_efficiency_with_beta.nc")
ds_hydropower_generation_merged = xr.merge([ds_hydropower_generation.rename({"gen": "hp_no_beta"}),
                                            ds_hydropower_generation_beta.rename({"gen": "hp_with_beta"})]).load()

df_hydropower_production_params = pd.read_csv(path_data_hydro / "hydropower_generation" / "ds_prevah_500_hydropower_production_parameters.csv")
df_hydropower_production_params.loc[df_hydropower_production_params["Expected yearly generation"] == 0, "Expected yearly generation"] = \
    df_hydropower_production_params[df_hydropower_production_params["Expected yearly generation"] == 0].apply(lambda row: round(row["Capacity"] * 365 * 24 * 1e-6, 2), axis=1)

## Load historical Swiss electricity generation data
df_historical_data = pd.read_csv(path_data / "energy" / "ogd35_schweizerische_elektrizitaetsbilanz_monatswerte.csv")

# Analyse data (without monthly bias correction)

In [ ]:
national_analysis_pre_correction = NationalAnalysisHydropower(gdf_switzerland, gdf_hydropower_polygons, df_wasta,
                                               ds_hydropower_generation_merged, 
                                               df_hydropower_production_params, 
                                               df_historical_data[["Jahr", "Monat", "Erzeugung_laufwerk_GWh", 
                                                                   "Erzeugung_speicherwerk_GWh", 
                                                                   "Verbrauch_speicherpumpen_GWh"]], 
                                               path_figs)

In [ ]:
national_analysis_pre_correction.aggregate_yearly_estimated_generation(with_percentage=False)
national_analysis_pre_correction.aggregate_yearly_estimated_generation_with_operation_start(with_percentage=False)
national_analysis_pre_correction.aggregate_yearly_estimated_generation_per_hp()
national_analysis_pre_correction.aggregate_yearly_estimated_generation_per_hp(reference_period=slice("1991", "2020"))

In [ ]:
national_analysis_pre_correction.aggregate_seasonal_estimated_generation(with_operation_start=False)
national_analysis_pre_correction.aggregate_seasonal_estimated_generation(with_operation_start=True)
national_analysis_pre_correction.aggregate_seasonal_estimated_generation(with_operation_start=False, per_hydropower=True)
national_analysis_pre_correction.aggregate_reference_seasonal_estimated_generation()

# Analyse data (with monthly bias correction)

In [ ]:
ds_monthly_bias_correction_factors = xr.open_dataset(path_data_hydro / "hydropower_generation" / "ds_prevah_500_hydropower_production_ror_simplified_efficiency_monthly_bias_correction_factors.nc").bias_correction_factor.load()

In [ ]:
national_analysis = NationalAnalysisHydropower(gdf_switzerland, gdf_hydropower_polygons, df_wasta,
                                               ds_hydropower_generation_merged * ds_monthly_bias_correction_factors, 
                                               df_hydropower_production_params, 
                                               df_historical_data[["Jahr", "Monat", "Erzeugung_laufwerk_GWh", 
                                                                   "Erzeugung_speicherwerk_GWh", 
                                                                   "Verbrauch_speicherpumpen_GWh"]], 
                                               path_figs)

In [ ]:
national_analysis.aggregate_yearly_estimated_generation(with_percentage=False)
national_analysis.aggregate_yearly_estimated_generation_with_operation_start(with_percentage=False)
national_analysis.aggregate_yearly_estimated_generation_per_hp()
national_analysis.aggregate_yearly_estimated_generation_per_hp(reference_period=slice("1991", "2020"))

In [ ]:
national_analysis.aggregate_seasonal_estimated_generation(with_operation_start=False)
national_analysis.aggregate_seasonal_estimated_generation(with_operation_start=True)
national_analysis.aggregate_seasonal_estimated_generation(with_operation_start=False, per_hydropower=True)
national_analysis.aggregate_reference_seasonal_estimated_generation()

# Paper plots

## Fig 2

In [ ]:
plot_pre_post_bias_correction_validation(
    national_analysis_pre_correction,
    national_analysis,
    with_percentage=False, 
    yearly_column_to_plot="Estimated Generation No Beta",
    winter_column_to_plot="Estimated Generation No Beta Winter",
    summer_column_to_plot="Estimated Generation No Beta Summer",
    subplots_titles=["Yearly", 
                     "Winter", 
                     "Summer"], 
    save=True, 
    output_filename="fig_2.pdf",
    file_format="pdf"
)

## Fig 3

In [ ]:
national_analysis.plot_quantile_maps(yearly=True, 
                                     variable_name="hp_no_beta",
                                     save=True,
                                     with_operation_start=True,
                                     with_decade_visualization=True,
                                     output_filename="fig_3.eps",
                                     file_format="eps"
                                    )

## Fig 4

In [ ]:
national_analysis.plot_trend_analysis(
        with_percentage=False, 
        yearly_column_to_plot="Estimated Generation No Beta",
        winter_column_to_plot="Estimated Generation No Beta Winter",
        summer_column_to_plot="Estimated Generation No Beta Summer",
        subplots_titles=["Yearly", "Winter", "Summer"],
        save=True,
        output_filename="fig_4.pdf",
        file_format="pdf"
    )

# Supplementary material

## Sup Fig 1 - Run-of-river hydropower in Switzerland

In [ ]:
national_analysis.plot_ror_map_capacities_hist(save=True, output_filename="sup_fig_1.pdf", file_format="pdf")

In [ ]:
ror_capacity_ch = national_analysis.gdf_hydropower_locations.loc[(national_analysis.gdf_hydropower_locations["WASTANumber"].isin(
            national_analysis.ds_hydropower_generation.hydropower.values
            )), "Capacity"].sum()

print(f"The capacity of RoR hydropower in Switzerland is currently {ror_capacity_ch} GW")

In [ ]:
national_analysis.gdf_hydropower_locations.loc[(national_analysis.gdf_hydropower_locations["WASTANumber"].isin(
            national_analysis.ds_hydropower_generation.hydropower.values
            )), "Capacity"].describe()

In [ ]:
ror_capacity_90th_percentile = national_analysis.gdf_hydropower_locations.loc[(national_analysis.gdf_hydropower_locations["WASTANumber"].isin(
            national_analysis.ds_hydropower_generation.hydropower.values
            )), "Capacity"].quantile(0.9)

ror_capacity_fraction_90th_percentile_and_above = national_analysis.gdf_hydropower_locations.loc[(national_analysis.gdf_hydropower_locations["WASTANumber"].isin(
            national_analysis.ds_hydropower_generation.hydropower.values
            )) & (national_analysis.gdf_hydropower_locations["Capacity"] >= ror_capacity_90th_percentile), "Capacity"].sum()/ror_capacity_ch

print(f"The RoR hydropower plants in Switzerland with capacities equal or above the 90th percentile constitute {round(ror_capacity_fraction_90th_percentile_and_above * 100, 2)}% of the total RoR capacity")

## Sup Fig 2 - RoR hydropower high production years

The **total number of high production years of hydropower plants (quantile >= 0.9) per decade** is decreasing, even with an expanding infrastructure.

In [ ]:
national_analysis.plot_hist_prod_quantile_threshold_per_decade(yearly=True, 
                                                               variable_name="hp_no_beta",
                                                               quantile_threshold=0.9,
                                                               higher_than=True,
                                                               with_operation_start=True,
                                                               output_filename="sup_fig_2.pdf",
                                                               save=True,
                                                               file_format="pdf")

## Sup Fig 3 & 4 - RoR hydropower generation trends

In [ ]:
national_analysis.plot_trend_analysis_per_month(variable_name="hp_no_beta", save=True, output_filename="sup_fig_3.pdf", file_format="pdf")

In [ ]:
national_analysis.plot_winter_trend_map_and_distribution(variable_name="hp_no_beta_winter", save=True, output_filename="sup_fig_4.pdf", file_format="pdf")